In [1]:
!pip install selenium
!pip install beautifulsoup4
!pip install feedparser newspaper3k waybackpy pandas
!pip install requests
!pip install "lxml[html_clean]"

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.4/9.4 MB 34.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 499.2/499.2 kB 12.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 52.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.3/81.3 kB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.1/211.1 kB 18.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 107.4/107.4 kB 10.0 MB/s eta 0:00:00
  Created wheel for tinysegmenter: filename=tinysegmenter-0.3-py3-none-any.whl size=13540 sha256=014b7c79ddff7b236921aa3da045e5fa0120ab92dcbb9f83e1773bafdbf0dab7
  Stored in directory: /root/.cache/pip/wheels/fc/ab/f8/cce3a9ae6d828bd346be695f7ff54612cd22b7cbd7208d68f3
  Created wheel for feedfinder2: filename=feedfinder2-0.0.4-py3-none-any.whl size=3341 sha

In [2]:
import feedparser
from newspaper import Article
import pandas as pd
import time
from datetime import datetime, timedelta
import os
import urllib.parse
import re
import requests
import tempfile

from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

# ✅ M7 종목 정의
m7_stocks = {
    "AAPL": "Apple",
    "MSFT": "Microsoft",
    "NVDA": "Nvidia",
    "TSLA": "Tesla",
    "AMZN": "Amazon",
    "META": "Meta",
    "GOOGL": "Google"
}

# ✅ 수집 기간 정의 -> 이부분 날짜 수정
start_date = datetime(2020, 1, 1)
end_date = datetime(2025, 4, 27)

# ✅ 뉴스 저장 리스트
all_articles = []

# ✅ RSS에서 원본 뉴스기사 URL 추출
def fetch_rss_links(query, from_date, to_date, max_links=20):
    query_full = f"{query} after:{from_date} before:{to_date}"
    query_encoded = urllib.parse.quote_plus(query_full)
    query_url = f"https://news.google.com/rss/search?q={query_encoded}&hl=en-US&gl=US&ceid=US:en"
    feed = feedparser.parse(query_url)

    links = []
    for entry in feed.entries[:max_links]:
        try:
            google_url = entry.link
            # 스킵: consent.google.com 관련 링크
            if "consent.google.com" in google_url:
                continue
            if "url=" in google_url:
                match = re.search(r"url=(https?[^&]+)", google_url)
                real_url = match.group(1) if match else google_url
            else:
                real_url = google_url
            links.append((real_url, entry.published, entry.title))
        except:
            continue
    return links

# ✅ Selenium을 사용하여 뉴스 본문 추출 (고유한 사용자 데이터 디렉터리 지정)
def extract_article_with_selenium(url):
    # ✅ 매번 임시 폴더 생성
    with tempfile.TemporaryDirectory() as tmpdirname:
        chrome_options = Options()
        chrome_options.add_argument("--headless")
        chrome_options.add_argument("--no-sandbox")
        chrome_options.add_argument("--disable-dev-shm-usage")
        chrome_options.add_argument(f"--user-data-dir={tmpdirname}")  # <- 고유한 폴더 사용

        # 크롬 드라이버 실행
        driver = webdriver.Chrome(options=chrome_options)
        driver.set_page_load_timeout(30)

        try:
            driver.get(url)

            # 페이지 로딩 기다리기
            WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.TAG_NAME, "body"))
            )
            time.sleep(2)  # 충분한 로딩 대기

            text = driver.find_element(By.TAG_NAME, "body").text
            return text

        except Exception as e:
            print(f"❌ Selenium error for URL: {url}\n{e}")
            return None

        finally:
            driver.quit()



# ✅ 수집 루프 시작
for ticker, name in m7_stocks.items():
    print(f"\n🔍 [{ticker}] Collecting news...")

    current_date = start_date
    while current_date < end_date:
        from_str = current_date.strftime("%Y-%m-%d")
        to_str = (current_date + timedelta(days=30)).strftime("%Y-%m-%d")

        links = fetch_rss_links(f"{name} stock", from_str, to_str, max_links=20)  # ⬅️ 상위 10개로 제한
        print(f"🗓️  {from_str} ~ {to_str}: {len(links)} links found")

        for url, pub_date, title in links:
            try:
                pub_dt = datetime.strptime(pub_date[:16], "%a, %d %b %Y")
                pub_str = pub_dt.strftime("%Y-%m-%d")
            except:
                continue

            article_text = extract_article_with_selenium(url)
            if not article_text or len(article_text) < 200:
                print(f"⚠️ Skipped: {url}")
                continue

            all_articles.append({
                "date": pub_str,
                "ticker": ticker,
                "company": name,
                "title": title,
                "text": article_text,
                "original_url": url
            })

            time.sleep(1)

        current_date += timedelta(days=30)

# ✅ 결과 저장 (하나의 CSV)
df = pd.DataFrame(all_articles)
os.makedirs("output", exist_ok=True)
# df.to_csv("output/M7_news_2020_2025.csv", index=False)
print(f"\n✅ Total articles collected: {len(df)}")
df


🔍 [AAPL] Collecting news...
🗓️  2020-01-01 ~ 2020-01-31: 20 links found


KeyboardInterrupt: 

In [ ]:
df # 데이터프레임 확인 후

In [ ]:
df.to_csv("output/M7_news_2020_2025.csv", index=False) # 마지막에 csv 파일로 저장
print("📁 Saved to: output/M7_news_2020_2025.csv")